In [1]:
%%capture
!pip install --upgrade google-api-python-client
!pip install google-auth
!pip install oauth2client
!pip install google-cloud
!pip install google-cloud-storage

You need to create credentials before you can use the scripts below ( see https://cloud.google.com/docs/authentication/getting-started)

In [3]:
import gcp as gcp

Help on module gcp:

NAME
    gcp

DESCRIPTION
    #import argparse
    #import googleapiclient

CLASSES
    builtins.object
        gpc
    
    class gpc(builtins.object)
     |  Methods defined here:
     |  
     |  __init__(self)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  create_instance(self, compute, project, zone, name, bucket)
     |      :param project:
     |      :param zone:
     |      :param name:
     |      :param bucket:
     |      :return:
     |  
     |  list_instances(self, compute, project, zone)
     |  
     |  test(self)
     |  
     |  wait_for_operation(self, compute, project, zone, operation)
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)

FILE
    /Users

In [ ]:
assert 1 == 0 

In [2]:
import argparse
import googleapiclient
from googleapiclient import discovery
from six.moves import input
from pprint import pprint
import subprocess
import json
import googleapiclient
from googleapiclient import discovery
from six.moves import input
import time
import os
import shlex
from google.cloud import storage
from google.cloud import *
path_to_key = '/Users/pgreisen/Projects/Enevolv/keys_gcp/ngsdataanalysis-08329a276003.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_key

In [3]:
try:
    from oauth2client.client import GoogleCredentials  # pylint: disable=g-import-not-at-top
    import google.auth.exceptions  # pylint: disable=g-import-not-at-top
    from googleapiclient import discovery  # pylint: disable=g-import-not-at-top
    from googleapiclient import errors  # pylint: disable=g-import-not-at-top
except ImportError:
    print("Warning, google-api-python-client or google-auth not installed, can't connect to GCP instances.")

In [4]:
try:
    credentials = GoogleCredentials.get_application_default()
except google.auth.exceptions.DefaultCredentialsError:
    print("No credentials set")

In [5]:
# initiate the resource manager to monitor projects
resourcemanager = discovery.build('cloudresourcemanager', 'v1', credentials=credentials)

In [6]:
# Use default credentials
credentials = GoogleCredentials.get_application_default()
# Build and initialize the API
compute_client = googleapiclient.discovery.build('compute', 'v1', credentials=credentials)

In [7]:
zone_list = []
# temp debug
project_id = 'ngsdataanalysis'
zone_result = compute_client.zones().list(project=project_id).execute()
for zone_rows in zone_result['items']:
    zone_list.append(zone_rows['name'])
    
    
print(zone_list)  

['us-east1-b', 'us-east1-c', 'us-east1-d', 'us-east4-c', 'us-east4-b', 'us-east4-a', 'us-central1-c', 'us-central1-a', 'us-central1-f', 'us-central1-b', 'us-west1-b', 'us-west1-c', 'us-west1-a', 'europe-west4-a', 'europe-west4-b', 'europe-west4-c', 'europe-west1-b', 'europe-west1-d', 'europe-west1-c', 'europe-west3-c', 'europe-west3-a', 'europe-west3-b', 'europe-west2-c', 'europe-west2-b', 'europe-west2-a', 'asia-east1-b', 'asia-east1-a', 'asia-east1-c', 'asia-southeast1-b', 'asia-southeast1-a', 'asia-southeast1-c', 'asia-northeast1-b', 'asia-northeast1-c', 'asia-northeast1-a', 'asia-south1-c', 'asia-south1-b', 'asia-south1-a', 'australia-southeast1-b', 'australia-southeast1-c', 'australia-southeast1-a', 'southamerica-east1-b', 'southamerica-east1-c', 'southamerica-east1-a', 'asia-east2-a', 'asia-east2-b', 'asia-east2-c', 'asia-northeast2-a', 'asia-northeast2-b', 'asia-northeast2-c', 'europe-north1-a', 'europe-north1-b', 'europe-north1-c', 'europe-west6-a', 'europe-west6-b', 'europe-we

In [8]:
# set further up in the code
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_key

# Instantiates a client
storage_client = storage.Client()

# List all the buckets available
for bucket in storage_client.list_buckets():
    print(bucket)

<Bucket: enevolv_ngs-1>


In [9]:
def create_instance(compute, project, zone, name, bucket):
    # Get the latest Debian Jessie image.
    ##image_response = compute.images().getFromFamily(
    ##    project='debian-cloud', family='debian-8').execute()
    ## Images can be identified here
    ## https://cloud.google.com/compute/docs/images#os-compute-support
    #image_response = compute.images().getFromFamily(
    #    project=project).execute()
    #source_disk_image = image_response['selfLink']
    ## , family='ubuntu-1404-lts'
    # Configure the machine
    machine_type = "zones/%s/machineTypes/n1-standard-1" % zone
    config = {
        'name': name,
        'machineType': machine_type,

        # Specify the boot disk and the image to use as a source.
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                'sourceImage': 'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/family/ubuntu-1404-lts'
                }
            }
        ],

        # Specify a network interface with NAT to access the public
        # internet.
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }]
    }
    return compute.instances().insert(
        project=project,
        zone=zone,
        body=config).execute()

In [10]:
def wait_for_operation(compute, project, zone, operation):
    print('Waiting for operation to finish...')
    while True:
        result = compute.zoneOperations().get(
            project=project,
            zone=zone,
            operation=operation).execute()

        if result['status'] == 'DONE':
            print("done.")
            if 'error' in result:
                raise Exception(result['error'])
            return result

        time.sleep(1)  

In [11]:
def list_instances(compute, project, zone):
    result = compute.instances().list(project=project, zone=zone).execute()
    return result['items']

In [12]:
compute = googleapiclient.discovery.build('compute', 'v1')
print('Creating instance.')
# Project
project = 'ngsdataanalysis'
# zone
zone = 'us-east1-b'
## Name to identify instance
name = 'ubuntu-1404-trusty-v20190429'
## Bucket - not defined for first run
backet = 'empty-stuff'

#compute.instances().start(project=project, zone=zone, instance=*).execute()
# instance_name = 'ubuntu-1404-trusty-v20190429'
instance_name = 'testpg'
operation = create_instance(compute, project, zone, instance_name, bucket)

Creating instance.


In [13]:
instance_name = name
wait_for_operation(compute, project, zone, operation['name'])

instances = list_instances(compute, project, zone)

print('Instances in project %s and zone %s:' % (project, zone))
for instance in instances:
    print(' - ' + instance['name'])
    print("""
Instance created.
It will take a minute or two for the instance to complete work.
Check this URL: http://storage.googleapis.com/{}/output.png
Once the image is uploaded press enter to delete the instance.
""".format(bucket))

Waiting for operation to finish...
done.
Instances in project ngsdataanalysis and zone us-east1-b:
 - testpg

Instance created.
It will take a minute or two for the instance to complete work.
Check this URL: http://storage.googleapis.com/<Bucket: enevolv_ngs-1>/output.png
Once the image is uploaded press enter to delete the instance.



In [14]:
assert 1 == 0

AssertionError: 

In [ ]:
    if wait:
        input()

    print('Deleting instance.')

    operation = delete_instance(compute, project, zone, instance_name)
    wait_for_operation(compute, project, zone, operation['name'])

def list_instances(compute, project, zone):
    result = compute.instances().list(project=project, zone=zone).execute()
    return result['items']


def create_instance(compute, project, zone, name, bucket):
    # Get the latest Debian Jessie image.
    image_response = compute.images().getFromFamily(
        project='debian-cloud', family='debian-8').execute()
    source_disk_image = image_response['selfLink']

    # Configure the machine
    machine_type = "zones/%s/machineTypes/n1-standard-1" % zone
    startup_script = open(
        os.path.join(
            os.path.dirname(__file__), 'startup-script.sh'), 'r').read()
    image_url = "http://storage.googleapis.com/gce-demo-input/photo.jpg"
    image_caption = "Ready for dessert?"

    config = {
        'name': name,
        'machineType': machine_type,

        # Specify the boot disk and the image to use as a source.
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                    'sourceImage': source_disk_image,
                }
            }
        ],

        # Specify a network interface with NAT to access the public
        # internet.
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }],

        # Allow the instance to access cloud storage and logging.
        'serviceAccounts': [{
            'email': 'default',
            'scopes': [
                'https://www.googleapis.com/auth/devstorage.read_write',
                'https://www.googleapis.com/auth/logging.write'
            ]
        }],

        # Metadata is readable from the instance and allows you to
        # pass configuration from deployment scripts to instances.
        'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script',
                'value': startup_script
            }, {
                'key': 'url',
                'value': image_url
            }, {
                'key': 'text',
                'value': image_caption
            }, {
                'key': 'bucket',
                'value': bucket
            }]
        }
    }

    return compute.instances().insert(
        project=project,
        zone=zone,
        body=config).execute()

def wait_for_operation(compute, project, zone, operation):
    print('Waiting for operation to finish...')
    while True:
        result = compute.zoneOperations().get(
            project=project,
            zone=zone,
            operation=operation).execute()

        if result['status'] == 'DONE':
            print("done.")
            if 'error' in result:
                raise Exception(result['error'])
            return result

        time.sleep(1)  

def delete_instance(compute, project, zone, name):
    return compute.instances().delete(
        project=project,
        zone=zone,
        instance=name).execute()

if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description=__doc__,
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument('project_id', help='Your Google Cloud project ID.')
    parser.add_argument(
        'bucket_name', help='Your Google Cloud Storage bucket name.')
    parser.add_argument(
        '--zone',
        default='us-central1-c',
        help='Compute Engine zone to deploy to.')
    parser.add_argument(
        '--name', default='cloud-demo-instance', help='New instance name.')

    args = parser.parse_args()

    main(args.project_id, args.bucket_name, args.zone, args.name)